In [7]:
import timeit
import os, psutil
import math
from heapq import *

#Setting Goal State as a Global Variable for ease of access
global goal_state
goal_state = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "0"]

#Setting Path as a Global Variable for ease of access
global path
path = []

#Declaring a Node Class that stores the State, Parent Node, Cost and the Action performed on Parent to reach State
class Node:
    def __init__(self,state,parent,action):
        self.state = state
        self.parent = parent
        self.action = action
        if self.parent is None:
            self.cost = 0
        else:
            self.cost = parent.cost + 1
    def __lt__(self, alternate):
        return (self.state.board < alternate.state.board) and (self.cost < alternate.cost)
    
    def __eq__(self,alternate):
        return self.state.board == alternate.state.board
    
    def __repr__(self):
        return str(self.state.board)

#Declaring Environment and Modification Function that helps identify Child Nodes
class Environment:
    def __init__(self,board):
        self.side = 4 
        self.board = board

    #Performs U, D, L, R Operations on Node to find Children Nodes
    def perform_action(self,action):
        new_board = self.board[:]
        zero_index = new_board.index('0')
        
        if action == "L" and (zero_index % 4)> 0:
            temp_L = new_board[zero_index - 1]
            new_board[zero_index - 1] = new_board[zero_index]
            new_board[zero_index] = temp_L
            
        if action == "R" and (zero_index % 4)< 3:
            temp_R = new_board[zero_index + 1]
            new_board[zero_index + 1] = new_board[zero_index]
            new_board[zero_index] = temp_R
            
        if action == "U" and (zero_index / 4)>= 1:
            temp_U = new_board[zero_index - 4]
            new_board[zero_index - 4] = new_board[zero_index]
            new_board[zero_index] = temp_U
            
        if action == "D" and (zero_index / 4)< 3:
            temp_D = new_board[zero_index + 4]
            new_board[zero_index + 4] = new_board[zero_index]
            new_board[zero_index] = temp_D
        
        return Environment(new_board)

#Manhattan Distance Heuristic Function that takes Node as Input and Outputs Manhattan Distance
def manhattan_distance(node):
    board = node.state.board
    side = node.state.side
    #Initialize Manhattan Distance as 0
    manhattan_distance = 0 
    instantaneous_distance = 0
    length_board = len(board)
    #Calculating Distance 
    for i in range(0, length_board):
        zero_index = int(board[i])
        if zero_index == 0:
            continue
        else:
            present_x = i / 4
            present_y = i % 4
            goal_x = (zero_index - 1) / 4
            goal_y = (zero_index - 1) % 4
            
            difference_x = abs(goal_x-present_x)
            difference_y = abs(goal_y-present_y)
            instantaneous_distance = instantaneous_distance + difference_x
            instantaneous_distance = instantaneous_distance + difference_y
            manhattan_distance = manhattan_distance + instantaneous_distance
    return manhattan_distance

#Misplaced Tiles Heuristic Function that takes Node as Input and Outputs Number of Misplaced Tiles
def misplaced_tiles(node):
    board = node.state.board
    side = node.state.side
    length_board = len(board)
    #Initialize Misplaced Tiles as 0
    misplaced_tiles = 0
    for i in range(1, length_board):
        if i == int(board[i-1]):
            continue
        if i != int(board[i-1]):
            misplaced_tiles = misplaced_tiles + 1
    return misplaced_tiles
    
#A-Star Search function that takes Initial node and Heuristic Function as Input
def astar_search(initial_node, heuristic):
    print("--------------- " +heuristic+ " ---------------")
    #Initial Time
    start_time = timeit.timeit() #Initialize Program Start Time
    #Initial Memory State during the Start of the Program
    Process = psutil.Process(os.getpid())
    initial_memory_state = Process.memory_info().rss / 1024.0
    #Declaring Initial Empty Lists
    queue_nodes = []
    visited_nodes = []
    #Heuristic Condition Checker
    if heuristic == "Manhattan Distance":
        heappush(queue_nodes, (initial_node.cost + manhattan_distance(initial_node), initial_node))
    elif heuristic == "Misplaced Tiles":
        heappush(queue_nodes, (initial_node.cost + misplaced_tiles(initial_node),  initial_node))
    #Move_Counter is used to Obtain the Number of Nodes Expanded
    move_counter = 0
    #While frontier is not empty:
    while(len(queue_nodes) > 0):
        current_node = heappop(queue_nodes)[1] # we only need node , not the cost
        move_counter = move_counter + 1
        visited_nodes.append(current_node)
        #Check if Goal State is Reached
        if(current_node.state.board == goal_state):
            path = []
            while(current_node.parent is not None):
                path.append(current_node.action)
                current_node = current_node.parent
            path.reverse()
            print("Moves: ", str(path))
            print("Number of Nodes expanded: ", move_counter)
             #Final Memory Usage State 
            final_memory_state = Process.memory_info().rss / 1024.0
            memory_used = final_memory_state - initial_memory_state
            print("Memory Used: " +str(memory_used)+  " Kilobytes")
            end_time = timeit.timeit()
            time_elapsed = end_time - start_time
            time_elapsed = round(time_elapsed*1000, 3)
            print("Time Taken: "+ str(time_elapsed) + " ms") #ms = milliseconds
            return path,move_counter
        else:
            #Children of Current Node are obtained
            children = []
            actions = ["U", "D", "L", "R"] # left,right, up , down ; actions define direction of movement of empty tile
            for action in actions:
                child_state = current_node.state.perform_action(action)
                child_node = Node(child_state,current_node,action)
                children.append(child_node)
            for child in children:
                #Checking for Previously Visited nodes
                if child in visited_nodes:
                    continue
                else:
                    if heuristic == "Manhattan Distance":
                        heappush(queue_nodes, (child.cost + manhattan_distance(child), child))
                    elif heuristic == "Misplaced Tiles":
                        heappush(queue_nodes, (child.cost + misplaced_tiles(child), child))
                    else:
                        print("Unknown Operation")
    print("Solution Could Not be Found")
    return False

def main():
    #Taking Input from User in the String Format
    initial_state = str(input("Enter the Initial State of the Puzzle: "))
    #Checking Initial Conditions 1 0 2 4 5 7 3 8 9 6 11 12 13 10 14 15
    if len(initial_state) < 37:
        print("Insufficient Data. Please Try Again")
        time.sleep(5)
        exit()
    #This combination of 15-puzzle cannot be computed
    if initial_state == "1 2 3 4 5 6 7 8 9 10 11 12 13 15 14 0":
        print("Impossible to Compute")
        time.sleep(5)
        exit()
    #Splitting the Input String Based on Whitespace
    initial_state = initial_state.split(" ")
    
    #Creating Initial Node from the Initial State (Obtained from User)
    initial_node = Node(Environment(initial_state), None, None)
    
    #Performing A-Star Search using Manhattan Distance Heuristic Measure
    Manhattan_output = astar_search(initial_node, "Manhattan Distance")
    #Performing A-Star Search using Number of Missing Tiles Heuristic Measure
    Misplaced_Tiles_output = astar_search(initial_node, "Misplaced Tiles")
    
if __name__=="__main__":
    main()

Enter the Initial State of the Puzzle: 1 0 3 4 5 2 6 8 9 10 7 11 13 14 15 12
--------------- Manhattan Distance ---------------
Moves:  ['D', 'R', 'D', 'R', 'D']
Number of Nodes expanded:  6
Memory Used: 0.0 Kilobytes
Time Taken: -2.105 ms
--------------- Misplaced Tiles ---------------
Moves:  ['D', 'R', 'D', 'R', 'D']
Number of Nodes expanded:  6
Memory Used: 0.0 Kilobytes
Time Taken: 1.577 ms
